## Import

In [9]:
# for read data
import os
import numpy as np
import pandas as pd

# model
from catboost import CatBoostClassifier

# Out of Fold
from sklearn.metrics import log_loss  
from sklearn.model_selection import StratifiedKFold

# for save
import joblib

## Read data

In [10]:
path = (os.path.abspath("./input"))

X_train = pd.read_csv(path +'/CAT_train.csv', encoding='cp949')
X_test = pd.read_csv(path +'/CAT_test.csv', encoding='cp949')
y_train = pd.read_csv(path +'/y_train.csv', encoding='cp949').group

In [11]:
train_ID, test_ID = X_train.custid, X_test.custid
del X_train['custid'], X_test['custid']

## Train & Out of Fold
Catboost는 Hyperparameter Tunning이 중요하지 않기에 학습횟수를 크게하여 학습시킨다.<br>
학습과 동시에 Submission data를 생성한다.

In [4]:
SKF = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)

In [5]:
model = CatBoostClassifier(iterations=1000, learning_rate = 0.03, bootstrap_type ='Bayesian',
                           objective='MultiClass', devices='0:1', task_type="GPU", random_state=0)

In [6]:
cat_pred = np.zeros((X_test.shape[0], 8))
loss_list = []
for tr_idx, val_idx in SKF.split(X_train, y_train):
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    model.fit(tr_x, tr_y, early_stopping_rounds = 1000)
    pred = model.predict_proba(val_x)
    loss = log_loss(val_y, pred)
    loss_list.append(loss)
    
    sub_pred = np.array(model.predict_proba(X_test)) / 5  # averaging
    cat_pred += sub_pred
print(f'{model.__class__.__name__}의 5fold 평균 Log Loss는 {np.mean(loss_list)}')

0:	learn: 2.0489011	total: 12.2ms	remaining: 12.1s
1:	learn: 2.0211395	total: 23ms	remaining: 11.5s
2:	learn: 1.9969228	total: 33.7ms	remaining: 11.2s
3:	learn: 1.9742064	total: 44.4ms	remaining: 11.1s
4:	learn: 1.9541596	total: 55.1ms	remaining: 11s
5:	learn: 1.9350281	total: 65.7ms	remaining: 10.9s
6:	learn: 1.9173171	total: 76.5ms	remaining: 10.9s
7:	learn: 1.9000471	total: 87.2ms	remaining: 10.8s
8:	learn: 1.8842902	total: 97.9ms	remaining: 10.8s
9:	learn: 1.8699362	total: 109ms	remaining: 10.8s
10:	learn: 1.8567105	total: 119ms	remaining: 10.7s
11:	learn: 1.8440009	total: 130ms	remaining: 10.7s
12:	learn: 1.8320504	total: 141ms	remaining: 10.7s
13:	learn: 1.8204024	total: 152ms	remaining: 10.7s
14:	learn: 1.8098423	total: 163ms	remaining: 10.7s
15:	learn: 1.8001882	total: 174ms	remaining: 10.7s
16:	learn: 1.7909382	total: 184ms	remaining: 10.7s
17:	learn: 1.7817685	total: 195ms	remaining: 10.6s
18:	learn: 1.7723347	total: 206ms	remaining: 10.6s
19:	learn: 1.7640019	total: 218ms	re

165:	learn: 1.4393045	total: 1.8s	remaining: 9.06s
166:	learn: 1.4385599	total: 1.81s	remaining: 9.05s
167:	learn: 1.4376093	total: 1.82s	remaining: 9.04s
168:	learn: 1.4367737	total: 1.84s	remaining: 9.03s
169:	learn: 1.4360635	total: 1.85s	remaining: 9.02s
170:	learn: 1.4351699	total: 1.86s	remaining: 9.01s
171:	learn: 1.4343115	total: 1.87s	remaining: 8.99s
172:	learn: 1.4334098	total: 1.88s	remaining: 8.98s
173:	learn: 1.4325224	total: 1.89s	remaining: 8.97s
174:	learn: 1.4317489	total: 1.9s	remaining: 8.96s
175:	learn: 1.4309082	total: 1.91s	remaining: 8.95s
176:	learn: 1.4299618	total: 1.92s	remaining: 8.94s
177:	learn: 1.4291521	total: 1.93s	remaining: 8.93s
178:	learn: 1.4282420	total: 1.94s	remaining: 8.91s
179:	learn: 1.4274129	total: 1.95s	remaining: 8.9s
180:	learn: 1.4265361	total: 1.97s	remaining: 8.89s
181:	learn: 1.4258250	total: 1.98s	remaining: 8.88s
182:	learn: 1.4251919	total: 1.99s	remaining: 8.87s
183:	learn: 1.4244277	total: 2s	remaining: 8.86s
184:	learn: 1.4235

333:	learn: 1.3357535	total: 3.61s	remaining: 7.2s
334:	learn: 1.3353043	total: 3.62s	remaining: 7.19s
335:	learn: 1.3348633	total: 3.63s	remaining: 7.18s
336:	learn: 1.3345034	total: 3.64s	remaining: 7.17s
337:	learn: 1.3341242	total: 3.65s	remaining: 7.15s
338:	learn: 1.3335858	total: 3.66s	remaining: 7.14s
339:	learn: 1.3332474	total: 3.67s	remaining: 7.13s
340:	learn: 1.3326067	total: 3.68s	remaining: 7.12s
341:	learn: 1.3320881	total: 3.69s	remaining: 7.11s
342:	learn: 1.3317069	total: 3.71s	remaining: 7.1s
343:	learn: 1.3312306	total: 3.72s	remaining: 7.09s
344:	learn: 1.3306488	total: 3.73s	remaining: 7.08s
345:	learn: 1.3301775	total: 3.74s	remaining: 7.07s
346:	learn: 1.3296753	total: 3.75s	remaining: 7.05s
347:	learn: 1.3292104	total: 3.76s	remaining: 7.04s
348:	learn: 1.3288257	total: 3.77s	remaining: 7.03s
349:	learn: 1.3284396	total: 3.78s	remaining: 7.02s
350:	learn: 1.3281245	total: 3.79s	remaining: 7.01s
351:	learn: 1.3276770	total: 3.8s	remaining: 7s
352:	learn: 1.3271

502:	learn: 1.2669031	total: 5.39s	remaining: 5.32s
503:	learn: 1.2664248	total: 5.4s	remaining: 5.31s
504:	learn: 1.2661380	total: 5.41s	remaining: 5.3s
505:	learn: 1.2658032	total: 5.42s	remaining: 5.29s
506:	learn: 1.2653594	total: 5.43s	remaining: 5.28s
507:	learn: 1.2650059	total: 5.44s	remaining: 5.27s
508:	learn: 1.2646327	total: 5.45s	remaining: 5.26s
509:	learn: 1.2642779	total: 5.46s	remaining: 5.25s
510:	learn: 1.2639103	total: 5.47s	remaining: 5.24s
511:	learn: 1.2635147	total: 5.48s	remaining: 5.22s
512:	learn: 1.2630183	total: 5.49s	remaining: 5.21s
513:	learn: 1.2625972	total: 5.5s	remaining: 5.2s
514:	learn: 1.2621381	total: 5.51s	remaining: 5.19s
515:	learn: 1.2617976	total: 5.53s	remaining: 5.18s
516:	learn: 1.2615896	total: 5.54s	remaining: 5.17s
517:	learn: 1.2611652	total: 5.55s	remaining: 5.16s
518:	learn: 1.2607660	total: 5.56s	remaining: 5.15s
519:	learn: 1.2605322	total: 5.57s	remaining: 5.14s
520:	learn: 1.2601256	total: 5.58s	remaining: 5.13s
521:	learn: 1.25

675:	learn: 1.2041489	total: 7.21s	remaining: 3.46s
676:	learn: 1.2037896	total: 7.22s	remaining: 3.44s
677:	learn: 1.2035095	total: 7.23s	remaining: 3.43s
678:	learn: 1.2030147	total: 7.24s	remaining: 3.42s
679:	learn: 1.2026448	total: 7.25s	remaining: 3.41s
680:	learn: 1.2022967	total: 7.26s	remaining: 3.4s
681:	learn: 1.2018076	total: 7.27s	remaining: 3.39s
682:	learn: 1.2015180	total: 7.28s	remaining: 3.38s
683:	learn: 1.2011326	total: 7.29s	remaining: 3.37s
684:	learn: 1.2007843	total: 7.3s	remaining: 3.36s
685:	learn: 1.2005453	total: 7.32s	remaining: 3.35s
686:	learn: 1.2002688	total: 7.33s	remaining: 3.34s
687:	learn: 1.2000153	total: 7.34s	remaining: 3.33s
688:	learn: 1.1997349	total: 7.35s	remaining: 3.32s
689:	learn: 1.1994053	total: 7.36s	remaining: 3.31s
690:	learn: 1.1992413	total: 7.37s	remaining: 3.29s
691:	learn: 1.1988351	total: 7.38s	remaining: 3.28s
692:	learn: 1.1986250	total: 7.39s	remaining: 3.27s
693:	learn: 1.1983880	total: 7.4s	remaining: 3.26s
694:	learn: 1.1

...

241:	learn: 1.3850506	total: 2.61s	remaining: 8.16s
242:	learn: 1.3844096	total: 2.62s	remaining: 8.15s
243:	learn: 1.3837839	total: 2.63s	remaining: 8.14s
244:	learn: 1.3831955	total: 2.64s	remaining: 8.13s
245:	learn: 1.3826127	total: 2.65s	remaining: 8.12s
246:	learn: 1.3819995	total: 2.66s	remaining: 8.11s
247:	learn: 1.3814096	total: 2.67s	remaining: 8.1s
248:	learn: 1.3806888	total: 2.68s	remaining: 8.09s
249:	learn: 1.3801762	total: 2.69s	remaining: 8.08s
250:	learn: 1.3796209	total: 2.7s	remaining: 8.06s
251:	learn: 1.3789889	total: 2.71s	remaining: 8.05s
252:	learn: 1.3784753	total: 2.73s	remaining: 8.05s
253:	learn: 1.3777596	total: 2.74s	remaining: 8.04s
254:	learn: 1.3771412	total: 2.75s	remaining: 8.02s
255:	learn: 1.3764298	total: 2.76s	remaining: 8.01s
256:	learn: 1.3757555	total: 2.77s	remaining: 8s
257:	learn: 1.3751951	total: 2.78s	remaining: 7.99s
258:	learn: 1.3747954	total: 2.79s	remaining: 7.98s
259:	learn: 1.3741222	total: 2.8s	remaining: 7.97s
260:	learn: 1.3735

411:	learn: 1.2987226	total: 4.41s	remaining: 6.3s
412:	learn: 1.2982273	total: 4.42s	remaining: 6.29s
413:	learn: 1.2978110	total: 4.43s	remaining: 6.28s
414:	learn: 1.2973474	total: 4.45s	remaining: 6.27s
415:	learn: 1.2970140	total: 4.46s	remaining: 6.25s
416:	learn: 1.2967644	total: 4.46s	remaining: 6.24s
417:	learn: 1.2962951	total: 4.48s	remaining: 6.23s
418:	learn: 1.2958948	total: 4.49s	remaining: 6.22s
419:	learn: 1.2953733	total: 4.5s	remaining: 6.21s
420:	learn: 1.2949976	total: 4.51s	remaining: 6.2s
421:	learn: 1.2945571	total: 4.52s	remaining: 6.19s
422:	learn: 1.2942511	total: 4.53s	remaining: 6.18s
423:	learn: 1.2938534	total: 4.54s	remaining: 6.17s
424:	learn: 1.2934926	total: 4.55s	remaining: 6.16s
425:	learn: 1.2931550	total: 4.56s	remaining: 6.14s
426:	learn: 1.2928665	total: 4.57s	remaining: 6.13s
427:	learn: 1.2924450	total: 4.58s	remaining: 6.12s
428:	learn: 1.2918454	total: 4.59s	remaining: 6.11s
429:	learn: 1.2914940	total: 4.6s	remaining: 6.1s
430:	learn: 1.291

582:	learn: 1.2317171	total: 6.21s	remaining: 4.45s
583:	learn: 1.2312699	total: 6.23s	remaining: 4.43s
584:	learn: 1.2309976	total: 6.24s	remaining: 4.42s
585:	learn: 1.2305552	total: 6.25s	remaining: 4.41s
586:	learn: 1.2300724	total: 6.26s	remaining: 4.4s
587:	learn: 1.2298072	total: 6.27s	remaining: 4.39s
588:	learn: 1.2294836	total: 6.28s	remaining: 4.38s
589:	learn: 1.2291807	total: 6.29s	remaining: 4.37s
590:	learn: 1.2288718	total: 6.3s	remaining: 4.36s
591:	learn: 1.2285974	total: 6.31s	remaining: 4.35s
592:	learn: 1.2282303	total: 6.32s	remaining: 4.34s
593:	learn: 1.2279137	total: 6.33s	remaining: 4.33s
594:	learn: 1.2274827	total: 6.34s	remaining: 4.32s
595:	learn: 1.2272188	total: 6.35s	remaining: 4.3s
596:	learn: 1.2267254	total: 6.36s	remaining: 4.29s
597:	learn: 1.2263549	total: 6.37s	remaining: 4.28s
598:	learn: 1.2260691	total: 6.38s	remaining: 4.27s
599:	learn: 1.2257993	total: 6.39s	remaining: 4.26s
600:	learn: 1.2254612	total: 6.4s	remaining: 4.25s
601:	learn: 1.22

749:	learn: 1.1727662	total: 7.97s	remaining: 2.66s
750:	learn: 1.1723840	total: 7.98s	remaining: 2.65s
751:	learn: 1.1721904	total: 7.99s	remaining: 2.64s
752:	learn: 1.1719356	total: 8s	remaining: 2.63s
753:	learn: 1.1716332	total: 8.01s	remaining: 2.61s
754:	learn: 1.1712701	total: 8.02s	remaining: 2.6s
755:	learn: 1.1709186	total: 8.03s	remaining: 2.59s
756:	learn: 1.1705078	total: 8.04s	remaining: 2.58s
757:	learn: 1.1699485	total: 8.06s	remaining: 2.57s
758:	learn: 1.1694446	total: 8.07s	remaining: 2.56s
759:	learn: 1.1691648	total: 8.08s	remaining: 2.55s
760:	learn: 1.1687724	total: 8.09s	remaining: 2.54s
761:	learn: 1.1685204	total: 8.1s	remaining: 2.53s
762:	learn: 1.1682108	total: 8.11s	remaining: 2.52s
763:	learn: 1.1678764	total: 8.12s	remaining: 2.51s
764:	learn: 1.1674852	total: 8.13s	remaining: 2.5s
765:	learn: 1.1671331	total: 8.14s	remaining: 2.49s
766:	learn: 1.1669690	total: 8.15s	remaining: 2.48s
767:	learn: 1.1665549	total: 8.16s	remaining: 2.46s
768:	learn: 1.1661

922:	learn: 1.1187144	total: 9.79s	remaining: 817ms
923:	learn: 1.1183597	total: 9.8s	remaining: 806ms
924:	learn: 1.1179909	total: 9.81s	remaining: 795ms
925:	learn: 1.1177135	total: 9.82s	remaining: 785ms
926:	learn: 1.1173652	total: 9.83s	remaining: 774ms
927:	learn: 1.1169266	total: 9.84s	remaining: 764ms
928:	learn: 1.1166828	total: 9.85s	remaining: 753ms
929:	learn: 1.1164319	total: 9.86s	remaining: 742ms
930:	learn: 1.1161424	total: 9.87s	remaining: 732ms
931:	learn: 1.1158119	total: 9.88s	remaining: 721ms
932:	learn: 1.1155599	total: 9.89s	remaining: 710ms
933:	learn: 1.1152567	total: 9.9s	remaining: 700ms
934:	learn: 1.1149101	total: 9.91s	remaining: 689ms
935:	learn: 1.1145775	total: 9.92s	remaining: 679ms
936:	learn: 1.1144291	total: 9.93s	remaining: 668ms
937:	learn: 1.1140548	total: 9.94s	remaining: 657ms
938:	learn: 1.1137669	total: 9.95s	remaining: 647ms
939:	learn: 1.1134697	total: 9.96s	remaining: 636ms
940:	learn: 1.1132750	total: 9.97s	remaining: 625ms
941:	learn: 1.

In [ ]:
# iterations = 1000, 1.494084167410715

## Make Submission File

In [7]:
pred = pd.DataFrame(cat_pred)
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
submissions = pd.concat([pd.Series(test_ID, name="ID"), pred] ,axis=1)

In [ ]:
sub_path = (os.path.abspath("./submission"))

fname = '/MCAT_FCAT.csv' # Model_FeatureSet
submissions.to_csv(sub_path+fname, index=False)
print("'{}' is ready to submit." .format(fname))